This notebook demonstrates basic usage of the firefly_client API for Firefly Slate Viewer to render tables, images and charts in a grid layout style. This notebook shows the same layout content as what ffapi-api-slate2.html renders in Firefly.  

Note that it may be necessary to wait for some cells (like those displaying an image) to complete before executing later cells.


## Setup

Imports for Python 2/3 compatibility
Imports for firefly_client

This example tentatively uses 127.0.0.1:8080 as the server
'slate.html' is a template made for grid view 

Open a browser to the firefly server in a new tab. Only works when running the notebook locally.

In [ ]:
from __future__ import print_function, division, absolute_import
from firefly_client import FireflyClient
import astropy.utils.data

host='127.0.0.1:8080'
channel = 'myChannel8'
html = 'slate.html'

fc = FireflyClient(host, channel=channel, html_file=html)
fc.launch_browser()

## Dispaly tables, images, and charts in Window/Grid like layout

Each rendered unit on Firefly Slate Viewer is called a'cell'. To render a cell and its content, the cell location (row, column, width, height), element type and cell ID are needed. (row, column) and (width, height) represent the position and size of the cell in terms of the grid blocks on Firefly Slate Viewer. Element types include types of 'tables', images', 'xyPlots', 'tableImageMeta' or 'coverageImage'. 

Entering a target to search the catalog and image

In [ ]:
target = '10.68479;41.26906;EQ_J2000'
# try
#target = '148.88822;69.06529;EQ_J2000'
# target = '202.48417;47.23056;EQ_J2000'
#



## Display tables, images and charts by using plotly


In [ ]:
fc.reinit_viewer()

# Add table in cell 'main'. 
# 'main' is the cell id currently supported by Firefly for element type 'tables'
# this cell is shown at row = 0, col = 0 with width = 4, height = 2

r = fc.add_cell(0, 0, 2, 2, 'tables', 'main')

if r['success']:
    fc.show_table(tbl_id='wiseCatTbl', title='another wise catalog', 
                  target_search_info={'catalogProject': 'WISE', 'catalog': 'allwise_p3as_psd',
                                'position': target, 'SearchMethod': 'Cone', 'radius': 1200},
                  options={'removable': True, 'showUnits': False, 'showFilters': True})
    
    
# Add first chart - scatter (plot.ly direct plot)
# in cell 0, 4, 2, 2, 
viewer_id = 'newChartContainer'
r = fc.add_cell(0, 4, 2, 2,'xyPlots', viewer_id)

if r['success']:
    trace1 = {'x': "tables::wiseCatTbl,ra",
              'y': "tables::wiseCatTbl,dec",
              'mode': 'markers',
              'type': 'scatter'}
    trace_data=[trace1]
    ffData=[{'dataType': 'fireflyScatter'}]
    
    fc.show_plot(group_id=viewer_id, data=trace_data, fireflyData=ffData  )
    
# Add second chart - heatmap  (plot.ly direct plot) 
# in cell 2, 0, 2, 3
viewer_id = 'heatMapContainer'
r = fc.add_cell(2, 0, 2, 3,'xyPlots', viewer_id)

if r['success']:
    dataHM = [
                {
                    'type': 'heatmap',
                    'name': 'w1-w2',
                    'x': "tables::wiseCatTbl,w1mpro",
                    'y': "tables::wiseCatTbl,w2mpro",
                    'colorscale': 'Blues'
                },
                {
                    'type': 'heatmap',
                    'name': 'w1-w3',
                    'x': "tables::wiseCatTbl,w1mpro",
                    'y': "tables::wiseCatTbl,w3mpro",
                    'colorscale': 'Reds',
                    'reversescale': True
                },
                {
                    'type': 'heatmap',
                    'name': 'w1-w4',
                    'x': "tables::wiseCatTbl,w1mpro",
                    'y': "tables::wiseCatTbl,w4mpro",
                    'colorscale': 'Greens'
                }]
    
    fireflyDataHM = [
                {'dataType': 'fireflyHeatmap'},
                {'dataType': 'fireflyHeatmap'},
                {'dataType': 'fireflyHeatmap'}]
    fc.show_plot(group_id=viewer_id, data=dataHM, fireflyData=fireflyDataHM )    
    
# Add third chart - histogram  (plot.ly direct plot) 
# in cell 2, 2, 2, 3
viewer_id =  'histContainer'
r = fc.add_cell(2, 2, 2, 3, 'xyPlots', viewer_id)

if r['success']:
    dataH = [
        {'name': 'dec-0.02', 'marker': {'color': 'rgba(153, 51, 153, 0.8)'}},
        {'name': 'dec+0.02', 'marker': {'color': 'rgba(102,153,0, 0.7)'}}
    ]
    
    fireflyDataH = [
        {
            'dataType': 'fireflyHistogram',
            'tbl_id': 'wiseCatTbl',
            'options': {
                'algorithm': 'fixedSizeBins',
                'fixedBinSizeSelection': 'numBins',
                'numBins': 30,
                'columnOrExpr': 'dec-0.02'}
        }, 
        {
            'dataType': 'fireflyHistogram',
            'tbl_id': 'wiseCatTbl',
            'options': {
                'algorithm': 'fixedSizeBins',
                'fixedBinSizeSelection': 'numBins',
                'numBins': 40,
                'columnOrExpr': 'dec+0.02'   
            }
        }
    ]
    result = fc.show_plot(group_id=viewer_id, data=dataH, fireflyData=fireflyDataH )  
    print(result)
    

# Add fourth chart - 3D plot (plot.ly direct plot)
# in cell 2, 4, 2, 3, 
viewer_id =  '3dChartContainer'
r = fc.add_cell(2, 4, 2, 3, 'xyPlots', viewer_id)

if r['success']:
    data3d = [
        {
            'type': 'scatter3d',
            'name': 'w1-w2',
            'x': "tables::wiseCatTbl,ra",
            'y': "tables::wiseCatTbl,dec",
            'z': "tables::wiseCatTbl,w1mpro",
            'mode' : 'markers',
            'marker' : {
                'size': 8,
                'line': {
                    'color': 'rgb(127, 127, 127, 0.14)',
                    'width': 1}
            },
            'hoverinfo': 'x+y+z'
        }]

    layout3d = {
        'xaxis': {
            'title': 'ra'
        },
        'yaxis': {
            'title': 'dec'
        },
        'zaxis': {
            'title': 'w1mpro'
        }
    }
    fc.show_plot(group_id=viewer_id, layout=layout3d, data=data3d )    
    
# Add a three color fits
# in cell 0, 2, 2, 2
viewer_id = '3C'
r = fc.add_cell(0, 2, 2, 2, 'images', viewer_id)

if r['success']:
    threeC= [
         {
             'Type'      : 'SERVICE',
             'Service'   : 'TWOMASS',
             'Title'     : '3 color',
             'SurveyKey'  : 'j',
             'WorldPt'    : target,
             'SizeInDeg'  : '.12'
         },
         {
             'Type'      : 'SERVICE',
             'Service'   : 'TWOMASS',
             'Title'     : '3 color',
             'SurveyKey'  : 'h',
             'WorldPt'    : target,
             'SizeInDeg'  : '.12'
         },
         {
             'Type'      : 'SERVICE',
             'Service'   : 'TWOMASS',
             'Title'     : '3 color',
             'SurveyKey'  : 'k',
             'WorldPt'    : target,
             'SizeInDeg'  : '.12'
         }]
        
    fc.show_fits_3color(threeC, viewer_id=viewer_id)

                        